# Overview Materi

Source: https://www.youtube.com/watch?v=LDRbO9a6XPU

Jelaskan secara singkat apa itu decision tree menurut pemahamanmu!
Sebuah metode prediksi dengan model berbentuk pohon yang dipakai untuk membuat keputusan atau klasifikasi data.

# Import Data & Libraries

In [1]:
from __future__ import print_function

# label kolom
header = ["color", "diameter", "label"]

# data training
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# data testing
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# Fungsi Dasar

In [8]:
# fungsi mencari apa saja unique value dari suatu kolom
def unique_values(rows, col):
    return list(set(row[col] for row in rows))

# contoh penggunaan
print(unique_values(training_data, 0))
print(unique_values(training_data, 1))
print(unique_values(training_data, 2))

['Green', 'Yellow', 'Red']
[1, 3]
['Apple', 'Grape', 'Lemon']


In [10]:
# fungsi Menghitung jumlah unique value dari suatu kolom
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

# contoh penggunaan
print(class_counts(training_data))
print(class_counts(testing_data))

{'Apple': 2, 'Grape': 2, 'Lemon': 1}
{'Apple': 2, 'Grape': 2, 'Lemon': 1}


In [18]:
# fungsi pengecekan suatu value numerik atau bukan
def is_numeric(value):
    try:
        float(value)
        return True
    except:
        return False
# contoh penggunaan
print(is_numeric(1))
print(is_numeric('Red')) # ini bukan angka jadi akan menghasilkan sebuah kesalahan (False)

True
False


In [30]:
# kelas untuk merepresentasikan pertanyaan pada decision tree
class Question:

    # inisialisasi kolom dan nilai pertanyaan
    def __init__(self, column, value):
      self.column = column
      self.value = value

    # mengecek apakah contoh data sesuai dengan pertanyaan
    def match(self, example):
      val = example[self.column]
      if is_numeric(val):
            return val >= self.value
      else:
            return val == self.value

    # menampilkan pertanyaan dalam format string yang mudah dibaca
    def __repr__(self):
      condition = "=="
      if is_numeric(self.value):
            condition = ">="
      return "Is {} {} {}?".format(
            header[self.column], condition, str(self.value))

# contoh penggunaan 1
q = Question(0, 'Green')
print(q)
print(q.match(training_data[0]))

q = Question(0, 3)
print(q)
print(q.match(training_data[0]))


Is color == Green?
True
Is color >= 3?
False


In [82]:
# membagi dataset menjadi dua berdasarkan pertanyaan
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

# contoh penggunaan
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
print(len(true_rows), len(false_rows))
false_rows

2 3


[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [83]:
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

**apa itu gini impurity?**
<br> gini impurity berfungsi mengukur tingkat ketidakmurnian atau ketidakteraturan pada sebuah simpul (node) dalam pohon

In [36]:
# menghitung nilai Gini Impurity untuk sebuah dataset
def gini(rows):
    counts = class_counts(rows)
    if len(rows) == 0:
        return 0
    impurity = 1
    for lbl in counts:
        prob = counts[lbl] / float(len(rows))
        impurity -= prob ** 2
    return impurity

# contoh penggunaan
print(gini(training_data))

0.6399999999999999


In [84]:
no_mixing = [['Apple'],
              ['Apple']]
gini(no_mixing)

0.0

**apa itu information gain?**
<br> information gain berfungsi mengukur seberapa efektif sebuah fitur dalam memisahkan data berdasarkan kelas-kelasnya

In [37]:
# menghitung nilai Information Gain dari pemisahan dataset
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

# contoh penggunaan
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
current = gini(training_data)
print(info_gain(true_rows, false_rows, current))

0.37333333333333324


In [39]:
# mencari pertanyaan terbaik untuk membagi dataset berdasarkan information gain tertinggi
def find_best_split(rows):
    best_gain = 0  # default
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1

    for col in range(n_features):
        values = set(row[col] for row in rows)
        for val in values:
            question = Question(col, val)


            # splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

# contoh penggunaan
best_gain, best_question = find_best_split(training_data)
print(best_gain, best_question)

0.37333333333333324 Is diameter >= 3?


# Fungsi Decision Tree

In [40]:
# merepresentasikan node daun (leaf) pada decision tree yang berisi hasil prediksi
class Leaf:

    # inisialisasi leaf dengan menghitung jumlah kemunculan tiap kelas
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [41]:
# merepresentasikan node keputusan (decision node) yang berisi pertanyaan dan cabang
class Decision_Node:

    # inisialisasi node dengan pertanyaan, cabang benar, dan cabang salah
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch


In [61]:
# membangun decision tree secara rekursif
def build_tree(rows):
    if len(rows) == 0:
        return Leaf([])
    current_uncertainty = gini(rows)
    if current_uncertainty == 0:
        return Leaf(rows)
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(question, true_branch, false_branch)



In [71]:
# mencetak struktur decision tree secara rekursif dalam format teks
def print_tree(node, spacing=""):

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return

    # mencetak pertanyaan pada node saat ini
    print(spacing + str(node.question))

    # mencetak cabang true secara rekursif
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # mencetak cabang false secara rekursif
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

# contoh penggunaan
my_tree = build_tree(training_data)
print_tree(my_tree)

Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


In [72]:
# mengklasifikasikan satu baris data menggunakan decision tree
def classify(row, node):

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        return node.predictions

    # menentukan apakah mengikuti cabang true atau cabang false
    # dengan membandingkan nilai fitur pada baris dengan pertanyaan di node
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

# contoh penggunaan
result = classify(testing_data[0], my_tree)
print(result)

{'Apple': 1}


In [78]:
# menampilkan prediksi pada leaf dalam format persentase
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts:
        probs[lbl] = str(counts[lbl] / total* 100) + "%"
    print(probs)



# contoh penggunaan
print_leaf(classify(testing_data[0], my_tree))

{'Apple': '100.0%'}


# Predict Using Decision Tree

In [79]:
# menguji decision tree dengan data uji dan membandingkan hasil prediksi dengan label asli
for row in testing_data:
    print("Actual: {}".format(row[-1]))
    print("Predicted:")
    result = classify(row, my_tree)
    print_leaf(result)
    print("-----")

Actual: Apple
Predicted:
{'Apple': '100.0%'}
-----
Actual: Apple
Predicted:
{'Apple': '50.0%', 'Lemon': '50.0%'}
-----
Actual: Grape
Predicted:
{'Grape': '100.0%'}
-----
Actual: Grape
Predicted:
{'Grape': '100.0%'}
-----
Actual: Lemon
Predicted:
{'Apple': '50.0%', 'Lemon': '50.0%'}
-----
